In [ ]:
import numpy as np

class Problem:
            
    def __init__(self):
        self.A = None
        self.B = None
        self.C = None
        self.b = None
        self.r = None
        
    def calc(self,x):
        return matvec(A,B,C,x)
    
    
    
def matvec(A,B,C,x):
    

def solve_linear(provlem):

    
def newton_solver(problem,x0,max_iter,eps,h,r):
    x = x0.copy()
    p = problem
    p.b+=r
    for i in range(max_iter):
        x += solve_linear(p)
        p = generate_problem(h,x)
        p.b+=r
        if(np.abs(p.calc(x)-b).mean() < eps):
            break
        
    return x    


def solver(problem,x0,h,max_h,r):
    eps = 1e-7
    max_iter=1000
    if(h >= max_h):
        return newton_solver(problem,x0,max_iter,eps,h,r)
    else:
        p = problem
        p.b+=r
        for i in range(max_iter):
            x = x0.copy()
            x = multigrid(p,h,x,min_h)
            p = generate_problem(h,x)
            p.b+=r
            if(np.abs(p.calc(x)-b).mean() < eps):
                return x
    
    
def interpolate(x):
    
    
    
def interpolate_up(x):

    

def generate_problem(h,x):
    
    


def multigrid(A,h,v,min_h):
    v2 = interpolate(v)
    A2 = generate_problem(2*h,v2)
    r2 = imterpolate(A.b-A.calc(v))
    u2 = solve(A2,v2,2*h,min_h,A2.calc(v2)+r2)
    u = v+interpolate_up(u2-v2)
    return u
    
    